In [ ]:
import numpy as np
import pandas as pd
import io
import re
import sys
!{sys.executable} -m pip install gensim
!{sys.executable} -m pip install nltk
import gensim
import nltk
nltk.download('punkt') #nltk Dependency
from gensim.models.doc2vec import Doc2Vec,TaggedDocument
from nltk.tokenize import word_tokenize
from datetime import datetime,date,timedelta


In [ ]:
newsDataframe = pd.read_csv('../input/newssampledata.csv')
marketDataframe = pd.read_csv('../input/marketsampledata.csv')

In [ ]:
newsDataframe.head()

In [ ]:
marketDataframe.head()

In [ ]:
%%time
inputMktData = pd.DataFrame()
inputMktData['date'] = marketDataframe['time']
inputMktData['date'] =  pd.to_datetime(inputMktData['date'])
inputMktData['date']=inputMktData['date'].dt.strftime('%d-%m-%Y')
inputMktData['date'] =  pd.to_datetime(inputMktData['date'])
inputMktData['assetName'] = marketDataframe['assetName']
inputMktData['volume'] = marketDataframe['volume']
inputMktData['close'] = marketDataframe['close']
inputMktData['open'] = marketDataframe['open']
inputNewsData = pd.DataFrame()
inputNewsData['date'] = newsDataframe['time']
inputNewsData['date'] =  pd.to_datetime(inputNewsData['date'])
inputNewsData['date']=inputNewsData['date'].dt.strftime('%d-%m-%Y')
inputNewsData['date'] =  pd.to_datetime(inputNewsData['date'])
inputNewsData['headline'] = newsDataframe['headline']
inputNewsData['urgency'] = newsDataframe['urgency']
inputNewsData['subjects'] = newsDataframe['subjects']
inputNewsData['audiences'] = newsDataframe['audiences']
inputNewsData['audiences'] = newsDataframe['audiences']
inputNewsData['assetName'] = newsDataframe['assetName']
inputNewsData['relevance'] = newsDataframe['relevance']
inputNewsData['sentimentNegative'] = newsDataframe['sentimentNegative']
inputNewsData['sentimentNeutral'] = newsDataframe['sentimentNeutral']
inputNewsData['sentimentPositive'] = newsDataframe['sentimentPositive']
del newsDataframe,marketDataframe


In [ ]:
inputNewsData = inputNewsData.dropna()

In [ ]:
inputNewsData.head()

In [ ]:
inputMktData.head()

In [ ]:
newsHeadlineData = inputNewsData['headline'].values

In [ ]:
%%time
tagged_data = [TaggedDocument(words=word_tokenize(str(_d).lower()), tags=[str(i)]) for i, _d in enumerate(newsHeadlineData)]
max_epochs = 5
vec_size = 25
alpha = 0.025
model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
model.build_vocab(tagged_data)
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    model.alpha -= 0.0002
    model.min_alpha = model.alpha

model.save("d2v.model")
del tagged_data
print("Model Saved")

In [ ]:
def longestVector(arr):
  k=-1
  maximum = -2
  for i in range(len(arr)):
    k = len(arr.item(i))
    if k>maximum:
      maximum = k
      index = i
  regex = r"[']([a-zA-z]*)[']"
  line = arr.item(index)
  matchObj = re.findall(regex, line, re.M|re.I)
  length = len(matchObj)
  return length
  

In [ ]:
def setOfAllCategories(arr):
  allCats = set()
  regex = r"[']([a-zA-z]*)[']"
  for i in range(len(arr)):
    line = arr.item(i)
    matchObj = re.findall(regex, line, re.M|re.I)
    for j in matchObj:
      allCats.add(j)
  return allCats

In [ ]:
def createDictVector(numCat,nors,vs,arr):
  vectorFrame = pd.DataFrame(np.zeros((nors,vs)))
  DictionaryOfCategory = dict(numCat)
  DictCat = {v: k for k, v in DictionaryOfCategory.items()}
  regex = r"[']([a-zA-z]*)[']"
  for i in range(len(arr)):
    line = arr.item(i)
    matchObj = re.findall(regex, line, re.M|re.I)
    k=0
    for j in matchObj:
      vectorFrame.at[i,k] = DictCat[j] 
      k = k+1
  df = vectorFrame
  df['features'] = df[df.columns.values].values.tolist()
  vectorNpArray = df['features'].values
  return vectorNpArray

In [ ]:
def vectorization(obj):
  print('Calculating the vector size')
  vectorSize = longestVector(obj)
  print('Creating a list of all unique category values')
  categories = setOfAllCategories(obj)
  categories = list(categories)
  numberOfRows = len(obj)
  print('Creating Numbered List of Categories')
  numberedListOfCategories = enumerate(categories)
  print('Making an array of feature')
  featureVector = createDictVector(numberedListOfCategories,numberOfRows,vectorSize,obj)
  print('Finished')
  return featureVector

In [ ]:
subjectArray = inputNewsData['subjects'].values
featureVectorSubject = vectorization(subjectArray)
del subjectArray
print(featureVectorSubject)

In [ ]:
audiencesArray = inputNewsData['audiences'].values
featureVectorAudience = vectorization(audiencesArray)
del audiencesArray
print(featureVectorAudience)

In [ ]:
def string2VecParser(obj):
  tempVector2 = []
  for i in range(len(obj)):
      stringConv = re.findall( r'([\d]*[.][\d]*)[,]', str(obj[i]))
      temp = []
      for j in stringConv:
        k = float(j)
        temp.append(k)
      tempVector = np.array(temp)
      tempVector2.append(tempVector)
  finalVector = np.array(tempVector2)
  return finalVector
  

In [ ]:
def queryEngine(marketIndex, newsDate):
  td = timedelta(days=1)
  upper = -45
  lower = 20
  p = inputMktData.loc[(inputMktData['date'] <= newsDate - upper*td) & (inputMktData['assetName'] == marketIndex) & (inputMktData['date'] >= newsDate - lower*td)]
  if p.empty:
    return pd.Series((None,None,None,None,None,None))
  labelFrameTemp = p[p['date'] >= newsDate]
  labelFrameTemp = labelFrameTemp.reset_index()
  if len(labelFrameTemp.index) < 5:
    return pd.Series((None,None,None,None,None,None))
  labelFrameTemp2 = pd.DataFrame()
  labelFrameTemp2['volume']=labelFrameTemp['volume']
  labelFrameTemp2['open']=labelFrameTemp['open']
  labelFrameTemp2['close']=labelFrameTemp['close']
  labelFrame = labelFrameTemp2.T
  labelFrame['labelTemp'] = labelFrame[labelFrame.columns.values].values.tolist()
  featureFrameTemp = p[p['date'] < newsDate]
  featureFrameTemp = featureFrameTemp.reset_index()
  featureFrameTemp2 = pd.DataFrame()
  featureFrameTemp2['volume']=featureFrameTemp['volume']
  featureFrameTemp2['open']=featureFrameTemp['open']
  featureFrameTemp2['close']=featureFrameTemp['close']
  featureFrame = featureFrameTemp2.T
  featureFrame['featureTemp'] = featureFrame[featureFrame.columns.values].values.tolist()
  label = labelFrame['labelTemp'].values
  feature = featureFrame['featureTemp'].values
  return pd.Series((label[0][:5],label[1][:5],label[2][:5],feature[0],feature[1],feature[2])) #label[volume open close] and feature[volume open close]

In [ ]:
def assembler(obj):
  assembledVector = pd.DataFrame()
  assembledVector['Vector'] = obj[obj.columns.values].values.tolist()
  return assembledVector

In [ ]:
def docvec(obj):
  obj = str(obj).lower()
  data = word_tokenize(obj)
  return model.infer_vector(data)

In [ ]:
def queryForPandas(row):
  return queryEngine(str(row['assetName']), row['date'])

In [ ]:
%%time
#news parsing
inputNewsData['subjectFeatures'] = featureVectorSubject.tolist() #subject to vector
print('Subject Vectorized...')
del featureVectorSubject
inputNewsData['audienceFeatures'] = featureVectorAudience.tolist() #audience to Vector
del featureVectorAudience
print('Audiences Vectorized...')
print('Headlines Vectorization Started')
inputNewsData['headlineVector'] = inputNewsData['headline'].apply(docvec) #headline to Vector
del model
print('Headlines Vectorized...')
print('Creating Cleaner DataFrame')
NewsDataframeFinal = pd.DataFrame()
NewsDataframeFinal['date'] = inputNewsData['date'] 
NewsDataframeFinal['urgency'] = inputNewsData['urgency']
NewsDataframeFinal['relevance'] = inputNewsData['relevance']
NewsDataframeFinal['assetName'] = inputNewsData['assetName']
NewsDataframeFinal['sentimentNegative'] = inputNewsData['sentimentNegative']
NewsDataframeFinal['sentimentNeutral'] = inputNewsData['sentimentNeutral']
NewsDataframeFinal['sentimentPositive'] = inputNewsData['sentimentPositive']
NewsDataframeFinal['subjectFeatures'] = inputNewsData['subjectFeatures']
NewsDataframeFinal['audienceFeatures'] = inputNewsData['audienceFeatures']
NewsDataframeFinal['headlineVector'] = inputNewsData['headlineVector']
del inputNewsData
print('DataFrame Creation Complete...')


In [ ]:
%%time
print('Query Engine Started...')
NewsDataframeFinal['labelVolume'] = np.nan
NewsDataframeFinal['labelOpen'] = np.nan
NewsDataframeFinal['labelClose'] = np.nan
NewsDataframeFinal['featureVolume'] = np.nan
NewsDataframeFinal['featureOpen'] = np.nan
NewsDataframeFinal['featureClose'] = np.nan
NewsDataframeFinal[['labelVolume','labelOpen','labelClose','featureVolume','featureOpen','featureClose']] = NewsDataframeFinal[['labelVolume','labelOpen','labelClose','featureVolume','featureOpen','featureClose']].astype(object)
NewsDataframeFinal[['labelVolume','labelOpen','labelClose','featureVolume','featureOpen','featureClose']] = NewsDataframeFinal.apply(queryForPandas, axis = 1)
NewsDataframeFinal = NewsDataframeFinal[NewsDataframeFinal.notnull()]
print('Query Engine Task Finished...')

In [ ]:
NewsDataframeFinal = NewsDataframeFinal[NewsDataframeFinal.notnull()]
NewsDataframeFinal = NewsDataframeFinal.dropna()
NewsDataframeFinal.head()

In [ ]:
def flat_list(obj):  
  listVal = [y for x in obj for y in x]
  return listVal

In [ ]:
Data = pd.DataFrame()
Data['label'] = NewsDataframeFinal[['labelVolume','labelOpen','labelClose']].values.tolist()
Data['sentimentFeature'] = NewsDataframeFinal[['urgency','relevance','sentimentPositive','sentimentNeutral','sentimentNegative']].values.tolist()
Data['newsFeature'] = NewsDataframeFinal[['subjectFeatures','audienceFeatures','headlineVector','featureVolume','featureOpen','featureClose']].values.tolist()
Data['newsFeature'] = Data['newsFeature'].apply(flat_list)
del NewsDataframeFinal
Data['features'] = Data[['sentimentFeature','newsFeature']].values.tolist()
Data = Data.drop(columns=['sentimentFeature','newsFeature'])
Data['label'] = Data['label'].apply(flat_list)
Data['features'] = Data['features'].apply(flat_list)

In [ ]:
Data.head()

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, Conv1D, MaxPooling1D, Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from sklearn.preprocessing import normalize


features = Data['features'].values
labels = Data['label'].values

In [ ]:
#hyperparameters
pool_size = 15
lstm_output_size = 15
batch_size = 7000
epochs = 5
max_features = 10000
maxlen = 200
embedding_size = 160

print('Loading data...')
x_train, x_test, y_train, y_test = train_test_split(features,labels, test_size= 0.2, random_state=21)

print('x_train shape:', len(x_train))
print('x_test shape:', len(x_test))
x_train = sequence.pad_sequences(x_train, maxlen=maxlen, padding='post')
x_test = sequence.pad_sequences(x_test, maxlen=maxlen, padding='post')
y_train = sequence.pad_sequences(x_train, maxlen=15, padding='post')
y_test = sequence.pad_sequences(x_test, maxlen=15, padding='post')
x_train = normalize(x_train)
y_train = normalize(y_train)
x_test = normalize(x_test)
y_test = normalize(y_test)

In [ ]:
print('Building model...')
model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen))
model.add(Dropout(0.5))
model.add(Conv1D (kernel_size = (5), filters = 40, activation='relu'))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(lstm_output_size))
model.add(Dropout(0.5))
model.add(Dense(15)) #prediction for next five days so the label should be made such that it has to be 15 dimensional(volume open close 5 days)
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
model.save('my_model.hdf5')
print('Test score:', score)
print('Test accuracy:', acc)
